In [16]:
"""Backend supported: tensorflow.compat.v1, tensorflow, pytorch"""
import deepxde as dde
import numpy as np

# General parameters
n = 2
precision_train = 10
precision_test = 10
hard_constraint = True
weights = 100  # if hard_constraint == False
epochs = 50000
parameters = [1e-3, 4, 80, "sin"]

# learning rate
# depth
# width
# activation function

# Define sine function
if dde.backend.backend_name == "pytorch":
    cos = dde.backend.pytorch.cos
else:
    from deepxde.backend import tf

    cos = tf.cos

learning_rate, num_dense_layers, num_dense_nodes, activation = parameters



In [ ]:

def pde(x, y):
    yRe, yIm = y[:, 0:1], y[:, 1:2]
    
    
    dyRe_xx = dde.grad.hessian(y, x, component=0, i=0, j=0)
    dyRe_yy = dde.grad.hessian(y, x, component=0, i=1, j=1)
    
    dyIm_xx = dde.grad.hessian(y, x, component=1, i=0, j=0)
    dyIm_yy = dde.grad.hessian(y, x, component=1, i=1, j=1)
    

    fRe = k0 ** 2 * cos(k0 * x[:, 0:1]) * cos(k0 * x[:, 1:2])
    fIm = k0 ** 2 * cos(k0 * x[:, 0:1]) * cos(k0 * x[:, 1:2])
    
    return [-dyRe_xx - dyRe_yy - k0 ** 2 * yRe - fRe,
            -dyIm_xx - dyIm_yy - k0 ** 2 * yIm - fIm]


# WORK / TODO /WORKING ON TRAVIS WHY
def func(x):
    real = np.cos(k0 * x[:, 0:1]) * np.cos(k0 * x[:, 1:2])
    imag = np.cos(k0 * x[:, 0:1]) * np.cos(k0 * x[:, 1:2])
    return np.hstack((real, imag))


def boundary(_, on_boundary):
    return on_boundary


geom = dde.geometry.Rectangle([0, 0], [1, 1])
k0 = 2 * np.pi * n
wave_len = 1 / n

hx_train = wave_len / precision_train
nx_train = int(1 / hx_train)

hx_test = wave_len / precision_test
nx_test = int(1 / hx_test)

bcRe = dde.icbc.NeumannBC(geom, lambda x: 0, boundary, component=0)
bcIm = dde.icbc.NeumannBC(geom, lambda x: 0, boundary, component=1)

bcs = [bcRe, bcIm]

data = dde.data.PDE(
    geom,
    pde,
    bcs,
    num_domain=nx_train ** 2,
    num_boundary=4 * nx_train,
    solution=func,
    num_test=nx_test ** 2,
)

net = dde.nn.FNN(
    [2] + [num_dense_nodes] * num_dense_layers + [2], activation, "Glorot uniform"
)

model = dde.Model(data, net)
loss_weights = [1, 1, weights, weights]
model.compile(
    "adam", lr=learning_rate, metrics=["l2 relative error"], 
    loss_weights=loss_weights
)


losshistory, train_state = model.train(epochs=epochs)
dde.saveplot(losshistory, train_state, issave=True, isplot=True)


Compiling model...
Building feed-forward neural network...
'build' took 0.046880 s



/usr/local/lib/python3.7/dist-packages/skopt/sampler/sobol.py:250: UserWarning: The balance properties of Sobol' points require n to be a power of 2. 0 points have been previously generated, then: n=0+402=402. 
  total_n_samples))
/usr/local/lib/python3.7/dist-packages/skopt/sampler/sobol.py:250: UserWarning: The balance properties of Sobol' points require n to be a power of 2. 0 points have been previously generated, then: n=0+84=84. 
  total_n_samples))
/usr/local/lib/python3.7/dist-packages/tensorflow/python/keras/legacy_tf_layers/core.py:171: UserWarning: `tf.layers.dense` is deprecated and will be removed in a future version. Please use `tf.keras.layers.Dense` instead.
  warnings.warn('`tf.layers.dense` is deprecated and '
/usr/local/lib/python3.7/dist-packages/tensorflow/python/keras/engine/base_layer_v1.py:1692: UserWarning: `layer.apply` is deprecated and will be removed in a future version. Please use `layer.__call__` method instead.
  warnings.warn('`layer.apply` is deprecate

'compile' took 1.244884 s

Initializing variables...
Training model...

Step      Train loss                                  Test loss                                   Test metric   
0         [9.05e+03, 7.36e+03, 7.29e+00, 3.43e+00]    [7.62e+03, 5.92e+03, 7.29e+00, 3.43e+00]    [1.08e+00]    
1000      [7.06e+03, 7.06e+03, 5.20e-02, 3.21e-02]    [5.63e+03, 5.63e+03, 5.20e-02, 3.21e-02]    [1.00e+00]    
2000      [7.06e+03, 7.06e+03, 3.52e-01, 3.49e-01]    [5.63e+03, 5.63e+03, 3.52e-01, 3.49e-01]    [1.00e+00]    
3000      [7.04e+03, 7.04e+03, 4.95e+00, 4.75e+00]    [5.65e+03, 5.65e+03, 4.95e+00, 4.75e+00]    [1.00e+00]    
4000      [2.65e+03, 2.65e+03, 3.18e+02, 3.24e+02]    [2.42e+03, 2.42e+03, 3.18e+02, 3.24e+02]    [1.03e+00]    
5000      [1.07e+01, 1.06e+01, 2.86e+00, 3.01e+00]    [1.23e+01, 1.15e+01, 2.86e+00, 3.01e+00]    [3.05e-01]    
6000      [2.63e+00, 2.66e+00, 5.88e-01, 6.49e-01]    [3.10e+00, 3.06e+00, 5.88e-01, 6.49e-01]    [1.55e-01]    
7000      [7.69e+00, 9.3